# New Algorithm
## by Zita Abreu, Joachim Rosenthal and Michael Schaller

In [1]:
from enum import Enum
from copy import deepcopy
import networkx as nx

%run state_transition_diagram.ipynb

class state_type(Enum):
    DEAD_FORWARD = 0
    DEAD_BACKWARD = 1
    ALIVE_FORWARD = 2
    ALIVE_BACKWARD = 3

def hamming_weight(matrix):
    weight = 0
    for i in range(matrix.nrows()):
        for j in range(matrix.ncols()):
            if matrix[i, j] != 0:
                weight += 1
    return weight
    
def dec_to_bin(number, length):
    # computes the binary representation of a decimal number in the way we need it
    # for the registers
    str_list = list(bin(number))[2:]
    bin_list = [0 for _ in range(length-len(str_list))] + [int(s) for s in str_list]
    bin_list.reverse()
    return matrix(1, length, bin_list)

def coefficients_up_to_deg(f, n):
    # returns the coefficients of a polynomial up to the coefficient of degree n.
    field = f.parent().base()
    coeff_list = f.coefficients(sparse=False)
    coeff_list = coeff_list + [0 for _ in range(n - len(coeff_list))]
    return matrix(field, n, 1, coeff_list)

def degree(generator_matrix):
    # works only for row reduced matrices.
    row_degs = row_degrees(generator_matrix)
    degree = sum(row_degs)
    return degree

def get_coefficient_matrices(generator_matrix, row_degs):
    # this function computes the matrices G_0,..., G_M from the generator matrix G(x)
    k = generator_matrix.nrows()
    n = generator_matrix.ncols()
    R = generator_matrix[0, 0].parent()
    x = R.0
    field = R.base()
    memory = max(row_degs)
    coeff_matrices = []
    for i in range(memory + 1):
        # for a polynomial f(x) = a_0 + a_1 x + ... + a_n x^n we get f(x) mod x^(i+1) = a_0 + ... + a_i x^i
        # therefore f(x) mod x^(i+1) - f(x) mod x^i = a_i x^i. Dividing by x^i we get a_i.
        # Similarly for the matrices.
        coeff_mat_poly = matrix(R, ((generator_matrix % x^(i+1)) - (generator_matrix % x^i)) / (x^i))
        coeff_mat = matrix(field, coeff_mat_poly)
        coeff_matrices.append(coeff_mat)
    return coeff_matrices

def make_sliding_matrix(generator_matrix, j, row_degs):
    n = generator_matrix.ncols()
    coefficient_matrices = get_coefficient_matrices(generator_matrix, row_degs)
    # make a long matrix (G_0,...,G_j)
    row_coefficient_matrices = block_matrix(1, j+1, coefficient_matrices[0:j+1])
    row_blocks = []
    for i in range(j+1):
        zero_mat = zero_matrix(generator_matrix.nrows(), n*i)
        # make the matrix (0, ..., 0, G_0,..., G_(j-i))
        row_block = block_matrix(1, 2, [[zero_mat, row_coefficient_matrices[:,:(j+1-i)*n]]], subdivide=False)
        row_blocks.append(row_block)
    # construct the sliding matrix from the matrices previously constructed.
    sliding_mat = block_matrix(j+1, 1, row_blocks, subdivide=False)
    return sliding_mat

def make_reverse_code_row(generator_matrix, memory):
    # for rate 1/n
    coeff_matrices = get_coefficient_matrices(generator_matrix, row_degs)
    memory = len(coeff_matrices) - 1
    generator_reverse = zero_matrix(coeff_matrices[0].nrows(), coeff_matrices[0].ncols())
    for i in range(memory + 1):
        # use the coefficient matrices in reverse order.
        generator_reverse += coeff_matrices[memory - i] * x^i
    return generator_reverse

def make_reverse_code(generator_matrix):
    k = generator_matrix.nrows()
    n = generator_matrix.ncols()
    row_degs = row_degrees(generator_matrix)
    rev_gen_mat_list = []
    for i in range(k):
        rev_gen_mat_list.append(make_reverse_code_row(matrix(1, n, generator_matrix[i]), row_degs[i]))
    return block_matrix(k, 1, rev_gen_mat_list)

def get_state_from_input_sequence(input_sequence, row_degs, j):
    k = len(row_degs)
    input_seq_rows = []
    for i in range(k):
        input_seq_rows.append([input_sequence[i+l*k] for l in range(j+1)])
    pseudo_state = [list(reversed(input_seq_rows[i])) for i in range(k)]
    state_list = []
    for i in range(k):
        if row_degs[i] == 0:
            state_list.append(())
        elif row_degs[i] < (j+1):
            state_list.append(tuple(pseudo_state[i][:row_degs[i]]))
        else:
            state_list.append(tuple(pseudo_state[i] + [0 for i in range(row_degs[i] - (j+1))]))
    return tuple(state_list)

def make_reverse_state(state):
    reverse_state_list = []
    k = len(state)
    for i in range(k):
        reverse_state_list.append(list(reversed(list(state[i]))))
    return tuple(reverse_state_list)

def modified_compute_distance_profile(generator_matrix, row_degs):
    # this is a bruteforce method
    field = generator_matrix[0, 0].parent().base()
    q = field.cardinality()
    k = generator_matrix.nrows()
    n = generator_matrix.ncols()
    memory = max(row_degs)
    delta = degree(generator_matrix)
    distance_profile = []
    distance_state_to_zero = {}
    zero_col = vector([0 for _ in range(k)])
    for j in range(memory + 1):
        # make the sliding matrix with first row block (G_0,..., G_j)
        sliding_matrix = make_sliding_matrix(generator_matrix, j, row_degs)
        # use the generalized Singleton bound as upper bound for the column distances.
        d_j = (n-k) * (delta//k + 1) + delta + 1
        # Consider all possible inputs. (This could be optimized since we also use inputs (u_0, ..., u_j)
        # with u_0 equal to the zero vector and we only later discard these inputs)
        # the input is a vector(u_0, ..., u_j) with u_i in F_q^k. Therefore the length is k*(j+1)
        len_input = k * (j + 1)
        # we make a list of lists of length len_input where each list contains all elements of the field.
        # This is used to generate all possible q^(k*(j+1)) inputs.
        inputs_to_be_combined = [[elt for elt in list(field)] for _ in range(len_input)]
        possible_inputs = list(itertools.product(*inputs_to_be_combined))
        for i in range(len(possible_inputs)):
            input_matrix = matrix(1, k*(j+1), possible_inputs[i])
            output = input_matrix * sliding_matrix
            # We do the following computation for all possible inputs instead of only the ones where
            # the u_0 is not the zero vector.
            # changes the complexity by 2 for k = 1 doing it like this for binary field.
            # for larger fields or k the relative workload changes by less.
            state_after_input = get_state_from_input_sequence(input_matrix[0], row_degs, j)
            if state_after_input in distance_state_to_zero.keys():
                dist_state = distance_state_to_zero[state_after_input]
                distance_state_to_zero[state_after_input] = min(dist_state, hamming_weight(output))
            else:
                distance_state_to_zero[state_after_input] = hamming_weight(output)
            if hamming_weight(output) < d_j and vector(input_matrix[0,:k]) != zero_col:
                d_j = hamming_weight(output)
        # append j-th column distance to distance profile
        distance_profile.append(d_j)
    return distance_profile, distance_state_to_zero

def adjust_stack(stack_list, W_0):
    adjusted_stack = []
    for S, W, m_vector in stack_list:
        if W - W_0 >= 0:
            # adjust the weight of a state by subtracting W_0.
            adjusted_stack.append((S, W - W_0, m_vector))
    return adjusted_stack

def determine_m_vector(state):
    m_vector = []
    for i in range(len(state)):
        m = 0
        if len(state[i]) == 0:
            m_vector.append(m)
            continue
        j = len(state[i]) - 1
        while state[i][j] == 0 and j >= 0:
            m += 1
            j -= 1
        m_vector.append(m)
    return m_vector

def new_m_vector(m_vector, inp):
    k = len(m_vector)
    m_vector_inp = [m_vector[i] + 1 if inp[i]== 0 else 0 for i in range(k)]
    return m_vector_inp

def max_row_degree_minus_m_vector(m_vector, row_degs):
    k = len(row_degs)
    return max([row_degs[i]- m_vector[i] for i in range(k)])

In [2]:
def combine_main_loop_after_extension(n, q, k, E, current_state, stored_states, diagram, row_degs, zero_state,
                                      W_star, final_round, distance_profile, distance_profile_reverse, memory):
    forward_type = [state_type.DEAD_FORWARD, state_type.ALIVE_FORWARD]
    backward_type = [state_type.DEAD_BACKWARD, state_type.ALIVE_BACKWARD]
    dead_type = [state_type.DEAD_FORWARD, state_type.DEAD_BACKWARD]
    alive_type = [state_type.ALIVE_FORWARD, state_type.ALIVE_BACKWARD]
    stored_states_copy = deepcopy(stored_states)
    W_m = current_state[2]
    T_m = current_state[1]
    S_m = current_state[0]
    W_f_m = W_star - W_m
    m_vector_m = current_state[3]
    state_count = 0
    zero_input = tuple(list(0 for _ in range(k)))
    
    if current_state[1] == state_type.ALIVE_FORWARD:
        # We look for the correct edge and determine the next state including the weight 
        new_state = diagram.nodes[S_m]["next state"][E]
        next_state_E = [
            new_state, state_type.ALIVE_FORWARD,
            hamming_weight(diagram[S_m][new_state][0]["output"]) + W_m,
            determine_m_vector(make_reverse_state(new_state))
        ]
        # The distance profile we are interested in is now the distance profile of the reverse code.
        dist_prof, dist_state_to_zero = distance_profile_reverse
    else:
        # We look for the correct edge and determine the next state including the weight
        # in this case we have to switch the arguments in diagram as we go from the previous state to S_m.
        new_state = diagram.nodes[S_m]["previous state"][E]
        next_state_E = [
            new_state, state_type.ALIVE_BACKWARD,
            hamming_weight(diagram[new_state][S_m][0]["output"]) + W_m,
            determine_m_vector(new_state)
        ]
        dist_prof, dist_state_to_zero = distance_profile
    # For convenience we put all the relevant parts of the next state into variables.
    S = next_state_E[0]
    W = next_state_E[2]
    W_f = W_star - next_state_E[2]
    m_vector = next_state_E[3]
    if S == zero_state:
        if W < W_star:
            W_star = W
        return stored_states_copy, W_star, state_count
    goto15 = False
    max_row_deg_minus_m = max_row_degree_minus_m_vector(m_vector, row_degs)
    # Step (6)
    # This is essentially a skip condition. It means that we don't need to consider this extension anymore.
    # The first part is the same as the usual one.
    # The other two conditions are the ones for FAST. The first one for zero input and not return to zero state.
    # The second one for non zero input.
    # These states don't need to be extended because because the column distances are too large.
    if (W > (W_star + n - 1) / 2 or 
        (W_f < dist_state_to_zero[S] and W_f < dist_prof[memory])
                or W_f < dist_prof[max_row_deg_minus_m-1]):
        goto15 = True
    goto9 = False
    # Step (7)
    if not goto15:
        for state in stored_states_copy:
            # update the stored states in the array.
            if state[0] == S:
                S_k = state[0]
                T_k = state[1]
                W_k = state[2]
                W_f_k = W_star - W_k
                if T_k in forward_type:
                    m_vector_k = determine_m_vector(make_reverse_state(S_k))
                else:
                    m_vector_k = determine_m_vector(S_k)
                goto9 = True
                break
    # Step (8)
    if not (goto9 or goto15):
        stored_states_copy.append([S, T_m, W, m_vector])
        state_count += 1
        goto15 = True
    goto13 = False
    # Step (9)
    if not goto15:
        if (T_k in forward_type and T_m in forward_type) or (T_k in backward_type and T_m in backward_type):
            goto13 = True
    # Step (10) and (11)
    if not (goto13 or goto15):
        if W + W_k < W_star:
            difference = W_star - (W + W_k)
            W_star = W + W_k
        if W >= W_k:
            goto15 = True
    # Step (12)
    if not (goto13 or goto15):
        W_k = W
        m_vector_k = m_vector
        if T_m in forward_type:
            if T_k in dead_type:
                T_k = state_type.DEAD_FORWARD
            else:
                T_k = state_type.ALIVE_FORWARD
        else:
            if T_k in dead_type:
                T_k = state_type.DEAD_BACKWARD
            else:
                T_k = state_type.ALIVE_BACKWARD          
        for state in stored_states_copy:
            if state[0] == S_k:
                state[2] = W_k
                state[1] = T_k
                state[3] = m_vector_k
                break
    # Step (13)
    if not goto15:
        if T_k in dead_type:
            goto15 = True
    # Step (14)
    if not goto15:
        if W < W_k:
            W_k = W
            m_vector_k = m_vector
        for state in stored_states_copy:
            if state[0] == S_k:
                state[2] = W_k
                state[3] = m_vector_k
                break
    # Step (15) and (16)
    if final_round:
        if T_m in forward_type:
            T_m = state_type.DEAD_FORWARD
        else:
            T_m = state_type.DEAD_BACKWARD
        for state in stored_states_copy:
            if state[0] == S_m:
                state[1] = T_m
                break
    return stored_states_copy, W_star, state_count

In [3]:
def combine_optimized_minimum_distance(generator_matrix, upper_bound="Singleton"):
    field = generator_matrix[0,0].parent().base()
    q = field.cardinality()
    polynomials = generator_matrix[0]
    k = generator_matrix.nrows()
    n = generator_matrix.ncols()
    try:
        row_degs, states = get_states(generator_matrix)
    except:
        raise
    memory = max(row_degs)
    delta = degree(generator_matrix)
    if upper_bound == "Singleton":
        upper_bound = (n-k) * (delta//k + 1) + delta + 1
    # make generator polynomial matrix into the g_i
    reverse_code_gen_mat = make_reverse_code(generator_matrix)
    distance_profile = modified_compute_distance_profile(generator_matrix, row_degs)
    distance_profile_reverse = modified_compute_distance_profile(reverse_code_gen_mat, row_degs)
    g_i_list = [coefficients_up_to_deg(f, memory + 1) for f in polynomials]
    # generate the matrix to get the outputs for each node.
    G_M_n = block_matrix(1, n, g_i_list, subdivide=False)
    state_transition_diagram, backward_diagram = make_diagram(generator_matrix, states, row_degs)
    # here the main part of the algorithm starts.
    W_star = upper_bound
    # generate all possible extension for the all zero state (forwards and backwards)
    zero_state = tuple([tuple([0 for _ in range(row_degs[i])]) for i in range(k)])
    starting_state = zero_state
    next_states_forward = state_transition_diagram[starting_state]
    final_state = zero_state
    next_states_backward = backward_diagram[final_state]
    # stored states is the array in the algorithm.
    # every element in the array consists of the state, the type and the weight of the state.
    # we do not want to consider the extension from the zero state to the zero state,
    # that's why we have the if condition.
    stored_states = []
    for new_state, multiedge in next_states_forward.items():
        wt_output = hamming_weight(multiedge[0]["output"])
        if new_state != zero_state:
            # This is to take the row degree 0 case into account.
            state_already_stored = False
            for stored_state in stored_states:
                if new_state == stored_state[0]:
                    state_already_stored = True
                    if wt_output < stored_state[2]:
                        stored_state = [new_state, state_type.ALIVE_FORWARD, wt_output,
                                        determine_m_vector(make_reverse_state(new_state))]
            if not state_already_stored:
                stored_states.append([new_state, state_type.ALIVE_FORWARD, wt_output,
                                     determine_m_vector(make_reverse_state(new_state))])
        else:
            # From zero state to zero state with nonzero input.
            if wt_output != 0:
                W_star = min(W_star, wt_output)
    for new_state_backward, multiedge_backward in next_states_backward.items():
        if new_state_backward != zero_state:
            weight_backward = hamming_weight(multiedge_backward[0]["output"])
            state_already_stored = False
            for stored_state in stored_states:
                if new_state_backward == stored_state[0]:
                    weight_path = weight_backward + stored_state[2]
                    if weight_path < W_star:
                        W_star = weight_path
                    state_already_stored = True
                    if weight_backward < stored_state[2]:
                        stored_state = [new_state_backward, state_type.ALIVE_BACKWARD, weight_backward,
                                       determine_m_vector(new_state_backward)]
            if not state_already_stored:
                stored_states.append([new_state_backward, state_type.ALIVE_BACKWARD, weight_backward,
                                     determine_m_vector(new_state_backward)])     
    finished = False
    current_state = None
    state_count = 1
    # start with step (2)
    while not finished:
        W_m = upper_bound
        # num_non_dead is to check condition (3) in the algorithm.
        num_non_dead = 0
        # We look for a non-dead state of minimal weight (Steps (2) and (3)).
        for state in stored_states:
            if state[1] == state_type.ALIVE_FORWARD or state[1] == state_type.ALIVE_BACKWARD:
                num_non_dead += 1
                if state[2] <= W_m:
                    W_m = state[2]
                    current_state = state
        W_m = current_state[2]
        T_m = current_state[1]
        S_m = current_state[0]
        # Step (3)
        if 2 * W_m >= W_star or num_non_dead <= 0:
            # Step (17)
            return W_star, state_count
        #make extensions.
        # at first we consider all possible inputs and then in the for loop we make the extension
        # according to the input and do the main part of the algorithm.
        possible_inputs = possibilities_inputs(generator_matrix, row_degs)
        for i in range(len(possible_inputs)):
            E = possible_inputs[i]
            # in the final round we have to make the current state dead.
            final_round = False
            if i == len(possible_inputs)-1:
                final_round = True
            stored_states, W_star, state_count_single_run = (
                combine_main_loop_after_extension(
                n, q, k, E, current_state, stored_states, state_transition_diagram, row_degs, zero_state,
                W_star, final_round, distance_profile, distance_profile_reverse, memory))
            state_count += state_count_single_run